In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%autoreload

In [3]:
import torch
import optuna
import os
import numpy as np
import pandas as pd
import deepdish as dd
import torch.nn as nn
import pytorch_lightning as pl

from torch.nn import functional as F
from pytorch_lightning import Callback
from pytorch_lightning.metrics.classification import F1
from optuna.integration import PyTorchLightningPruningCallback

In [9]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, PredefinedSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, roc_curve, auc, matthews_corrcoef, make_scorer

In [4]:
train = dd.io.load('../../splits/train.h5')
val = dd.io.load('../../splits/val.h5')
test = dd.io.load('../../splits/test.h5')
y_train = np.load('../../splits/y_train.npy')
y_val = np.load('../../splits/y_val.npy')
y_test = np.load('../../splits/y_test.npy')

In [5]:
y_train = y_train * 100
y_val = y_val * 100
y_test = y_test * 100

In [6]:
df_feat = pd.read_csv('../../datasets/dataset_extracted_features.csv')

In [7]:
df_feat

,Seq1,Seq2,Aln,GC1,GC2,SingleCon1,SingleCon2,PairCon1,PairCon2,SingleMFE1,SingleMFE2,Yield,Label
0,AGTACAAGTAGGACAGGAAGATA,TATCTTCCTGTCCTACTCGTACT,106,39.130435,43.478261,0.999957,0.999974,0.000022,0.000013,0.000000,0.000000,0.974025,1
1,GAGTTCCGGTTGCCTTTCA,GAGTTCCGGGTGCCTTTCA,12,52.631579,57.894737,0.999844,0.999739,0.000078,0.000130,0.000000,0.000000,0.000193,0
2,TCTGGAACTAGTGCAATTTAGC,TCTAAATTGCACTAGTTCCAGA,101,40.909091,36.363636,0.999556,0.999598,0.000222,0.000201,0.000000,0.000000,0.991153,1
3,GGCGCGCGACGCGACATCCGATAAGA,TCTTATCGGATCTCGCGTCGCGCGCC,121,65.384615,65.384615,0.999455,0.999183,0.000273,0.000408,-2.595847,-1.870977,0.999227,1
4,GTGTACCATACCATGAACGCCGGA,TCCGGCGTCATGGTATGGTACAC,110,54.166667,56.521739,0.998893,0.999652,0.000554,0.000174,0.000000,0.000000,0.996585,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2556971,AGCAAGGCACAGGACTTAGTACCAAA,TTTGGTACTAAGTCCTGTGCCGCT,113,46.153846,50.000000,0.999938,0.999916,0.000031,0.000042,-0.317577,0.000000,0.995240,1
2556972,CAGGCTGAGACCTAACTCTTTAGCC,GGCTAAAAGGTTAGGTCTCAGCCTG,110,52.000000,52.000000,0.999786,0.999896,0.000107,0.000052,0.000000,0.000000,0.973733,1
2556973,CATATCAACTCCAGGAATCCTT,AAGGATTCCTGCAGATGATATG,92,40.909091,40.909091,0.999334,0.998677,0.000333,0.000662,0.000000,0.000000,0.529762,1
2556974,GAATGCAGAGTTCGATGTCGGT,ACCAGTACACATCGAACTCTGCATTC,96,50.000000,46.153846,0.999855,0.999893,0.000073,0.000053,0.000000,0.000000,0.984665,1


In [8]:
train_features = df_feat.loc[(df_feat['Seq1'].isin([item[0] for item in train])) & (df_feat['Seq2'].isin([item[1] for item in train]))]
val_features = df_feat.loc[(df_feat['Seq1'].isin([item[0] for item in val])) & (df_feat['Seq2'].isin([item[1] for item in val]))]
test_features = df_feat.loc[(df_feat['Seq1'].isin([item[0] for item in test])) & (df_feat['Seq2'].isin([item[1] for item in test]))]

In [10]:
train_feat = train_features[['Aln', 'GC1', 'GC2', 'SingleCon1', 'SingleCon2', 'PairCon1', 'PairCon2', 'SingleMFE1', 'SingleMFE2']].values
val_feat = val_features[['Aln', 'GC1', 'GC2', 'SingleCon1', 'SingleCon2', 'PairCon1', 'PairCon2', 'SingleMFE1', 'SingleMFE2']].values
test_feat = test_features[['Aln', 'GC1', 'GC2', 'SingleCon1', 'SingleCon2', 'PairCon1', 'PairCon2', 'SingleMFE1', 'SingleMFE2']].values

In [11]:
train_labels = train_features[['Label']].values.squeeze()
val_labels = val_features[['Label']].values.squeeze()
test_labels = test_features[['Label']].values.squeeze()

In [12]:
scaler = MinMaxScaler()
scaler = scaler.fit(np.concatenate((train_feat, val_feat, test_feat), axis=0))
scaled_train = scaler.transform(train_feat)
scaled_val = scaler.transform(val_feat)
scaled_test = scaler.transform(test_feat)

In [13]:
torch.set_default_tensor_type(torch.FloatTensor)
torch.set_default_dtype(torch.float)

In [14]:
class MetricsCallback(Callback):
    """PyTorch Lightning metric callback."""

    def __init__(self):
        super().__init__()
        self.metrics = []

    def on_validation_end(self, trainer, pl_module):
        self.metrics.append(trainer.callback_metrics)

In [15]:
# Code taken and adapted from https://github.com/optuna/optuna/blob/master/examples/pytorch_lightning_simple.py
class LinearClassifier(nn.Module):
    def __init__(self, trial):
        super(LinearClassifier, self).__init__()
        self.layers = []
        self.dropouts = []

        # We optimize the number of layers, hidden units in each layer and dropouts.
        n_layers = trial.suggest_int("n_layers", 1, 5)
        dropout = trial.suggest_float("dropout", 0.2, 0.5)
        input_dim = 9
        for i in range(n_layers):
            output_dim = trial.suggest_int("n_units_l{}".format(i), 4, 128, log=True)
            self.layers.append(nn.Linear(input_dim, output_dim))
            self.dropouts.append(nn.Dropout(dropout))
            input_dim = output_dim

        # Binary classification layer
        self.layers.append(nn.Linear(input_dim, 1))

        # Assigning the layers as class variables (PyTorch requirement).
        # Parameters of a layer are returned when calling model.parameters(),
        # only if the layer is a class variable. Thus, assigning as class
        # variable is necessary to make the layer parameters trainable.
        for idx, layer in enumerate(self.layers):
            setattr(self, "fc{}".format(idx), layer)

        # Assigning the dropouts as class variables (PyTorch requirement), for
        # the same reason as above.
        for idx, dropout in enumerate(self.dropouts):
            setattr(self, "drop{}".format(idx), dropout)
            
    def forward(self, x):
        for layer, dropout in zip(self.layers, self.dropouts):
            x = F.relu(layer(x))
            x = dropout(x)
        return self.layers[-1](x)

In [16]:
class LightningWrapper(pl.LightningModule):
    def __init__(self, trial):
        super(LightningWrapper, self).__init__()
        self.trial = trial
        self.model = LinearClassifier(trial)
        self.batch_size = self.trial.suggest_categorical("batch_size", [256, 512, 1024])

    def forward(self, x):
        return self.model(x.float()).squeeze()

    def training_step(self, batch, batch_nb):
        x, target = batch
        output = self.forward(x)
        loss = F.binary_cross_entropy_with_logits(output, target.float())

        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_nb):
        x, target = batch
        output = self.forward(x)
        loss = F.binary_cross_entropy_with_logits(output, target.float())
        
        self.log("val_loss", loss)
        return loss
    
    def test_step(self, batch, batch_nb):
        x, target = batch
        output = self.forward(x)
        loss = F.binary_cross_entropy_with_logits(output, target.float())
        
        self.log("test_loss", loss)
        return loss

    def configure_optimizers(self):
        lr = self.trial.suggest_loguniform("lr", 1e-4, 1e-1)
        return torch.optim.Adam(self.model.parameters(), lr)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            list(zip(scaled_train, train_labels)),
            batch_size=self.batch_size,
            shuffle=True,
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            list(zip(scaled_val, val_labels)),
            batch_size=self.batch_size,
            shuffle=False,
        )
    
    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            list(zip(scaled_test, test_labels)),
            batch_size=self.batch_size,
            shuffle=False,
        )

In [17]:
SAVE_DIR = 'linear_nn_saved_models/'

In [18]:
def objective(trial):
    # Filenames for each trial must be made unique in order to access each checkpoint.
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        os.path.join(SAVE_DIR, "trial_{}".format(trial.number), "{epoch}"), monitor="val_loss"
    )

    # The default logger in PyTorch Lightning writes to event files to be consumed by
    # TensorBoard. We don't use any logger here as it requires us to implement several abstract
    # methods. Instead we setup a simple callback, that saves metrics from each validation step.
    metrics_callback = MetricsCallback()
    trainer = pl.Trainer(
        logger=False,
        checkpoint_callback=checkpoint_callback,
        gpus=[0],
        max_epochs=5,
        callbacks=[metrics_callback, PyTorchLightningPruningCallback(trial, monitor="val_loss")],
    )

    model = LightningWrapper(trial)
    trainer.fit(model)

    return metrics_callback.metrics[-1]["val_loss"].item()

In [19]:
pruner = optuna.pruners.HyperbandPruner()

study = optuna.create_study(direction="minimize", pruner=pruner)
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2021-02-15 18:00:41,523] A new study created in memory with name: no-name-f19f25a8-3ae8-4744-a355-5dfc7ba3878a
C:\Users\david\miniconda3\envs\pt1.8-cuda11.1\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/50 [00:00<?, ?it/s]

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | LinearClassifier | 5.0 K 
-------------------------------------------
5.0 K     Trainable params
0         Non-trainable params
5.0 K     Total params


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

[I 2021-02-15 19:32:49,132] Trial 49 pruned. Trial was pruned at epoch 3.
Number of finished trials: 50
Best trial:
  Value: 0.08271298557519913
  Params: 
    n_layers: 4
    dropout: 0.32964090436497967
    n_units_l0: 117
    n_units_l1: 18
    n_units_l2: 7
    n_units_l3: 19
    batch_size: 1024
    lr: 0.00021889729263244634
